In [1]:
import os

In [2]:
!pwd

/projects/templeton/analyses/201811/notebooks


In [3]:
#set pathnames
seq_filt = os.path.abspath('../tables/filtered/merged-repseqs.in-map.qza')
otu_filt_mitochloro = os.path.abspath('../tables/filtered/merged-table.in-map.nomito-nochloro.qza')
gg_ref_seq = os.path.abspath('/home/ssong/bin/gg_13_5_otus/rep_set/gg_13_5_otu_99.qza')
host_md_5per_fp = os.path.abspath('../../../01-metadata/5per_10k.11.28.18.short.txt')

In [4]:
!qiime feature-table filter-seqs \
  --i-data {seq_filt} \
  --i-table {otu_filt_mitochloro} \
  --o-filtered-data '../tables/filtered/merged-repseqs.in-map.nomito-nochloro.qza'

Saved FeatureData[Sequence] to: ../tables/filtered/merged-repseqs.in-map.nomito-nochloro.qza


In [6]:
!echo "qiime vsearch cluster-features-closed-reference \
  --i-sequences /projects/templeton/analyses/201811/tables/filtered/merged-repseqs.in-map.nomito-nochloro.qza \
  --i-table {otu_filt_mitochloro} \
  --i-reference-sequences {gg_ref_seq} \
  --p-perc-identity 1 \
  --p-threads 16 \
  --output-dir /projects/templeton/analyses/201811/tables/closed_ref_gg13_5" \
 | qsub -k eo -N temp_cr -l nodes=1:ppn=16 -l mem=64gb -l walltime=20:00:00 -V

1082057.barnacle.ucsd.edu


In [7]:
!echo "qiime feature-table rarefy \
  --i-table /projects/templeton/analyses/201811/tables/closed_ref_gg13_5/clustered_table.qza \
  --p-sampling-depth 5000 \
  --o-rarefied-table /projects/templeton/analyses/201811/tables/closed_ref_gg13_5/clustered_table.d5k.qza" \
 | qsub -k eo -N rarefy -l pmem=4gb -l walltime=10:00:00 -V

1082069.barnacle.ucsd.edu


In [19]:
!qiime feature-table filter-samples \
  --i-table '../tables/closed_ref_gg13_5/clustered_table.d5k.qza' \
  --m-metadata-file {host_md_5per_fp} \
  --o-filtered-table '../tables/closed_ref_gg13_5/clustered_table.d5k.5per.qza'

Saved FeatureTable[Frequency] to: ../tables/closed_ref_gg13_5/clustered_table.d5k.5per.qza


In [20]:
!qiime tools export \
  --input-path '../tables/closed_ref_gg13_5/clustered_table.d5k.5per.qza' \
  --output-path '../tables/closed_ref_gg13_5/cr_table'

Exported ../tables/closed_ref_gg13_5/clustered_table.d5k.5per.qza as BIOMV210DirFmt to directory ../tables/closed_ref_gg13_5/cr_table


In [17]:
!source deactivate

In [18]:
!source activate picrust

In [12]:
!mkdir ../picrust

In [14]:
!normalize_by_copy_number.py \
 -i '../tables/closed_ref_gg13_5/cr_table.d5k.biom' \
 -o '../picrust/cr_d5k.normalized_otus.biom'

In [15]:
!echo "predict_metagenomes.py \
 -i /projects/templeton/analyses/201811/picrust/cr_d5k.normalized_otus.biom \
 -o /projects/templeton/analyses/201811/picrust/cr_d5k.metagenome_predictions.biom \
 -a /projects/templeton/analyses/201811/picrust/cr_d5k.nsti_per_sample.txt" \
| qsub -oe -N picrust -l nodes=1:ppn=8 -l mem=32gb -l walltime=24:00:00 -V

1082076.barnacle.ucsd.edu


In [16]:
!categorize_by_function.py \
 -i '../picrust/cr_d5k.metagenome_predictions.biom' \
 -c KEGG_Pathways -l 3 \
 -o '../picrust/cr_d5k.metagenome_predictions.L3.biom'

!categorize_by_function.py \
 -i '../picrust/cr_d5k.metagenome_predictions.biom' \
 -c KEGG_Pathways -l 2 \
 -o '../picrust/cr_d5k.metagenome_predictions.L2.biom'

use cr_table.d5k.biom to run Bugbase (https://bugbase.cs.umn.edu/upload.html)